# Data Description
The sensors were sampledat the highest rate each device would support, and we segment thisdata into non-overlapping windows of 128 time steps. We includethe data collected from the 31 smartphones in our experiments.

# Data Processing

In [1]:
import numpy as np
import torch 
import os
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas as pd

scaler = StandardScaler()


In [ ]:
# from IPython.core.debugger import Tracer
# debug_here = Tracer()

In [2]:
data_dir = '../Activity recognition exp'
# HHAR_phones = pd.read_csv(f'{data_dir}/Phones_accelerometer.csv')
# HHAR_watch_ref = pd.read_csv(f'{data_dir}/Watch_accelerometer.csv')
HHAR_watch_ref = pd.read_csv(f'{data_dir}/Phones_accelerometer.csv')

HHAR_watch=HHAR_watch_ref.copy()

In [3]:
HHAR_watch.head(10)


,Index,Arrival_Time,Creation_Time,x,y,z,User,Model,Device,gt
0,0,1424696633908,1424696631913248572,-5.958191,0.688065,8.135345,a,nexus4,nexus4_1,stand
1,1,1424696633909,1424696631918283972,-5.952240,0.670212,8.136536,a,nexus4,nexus4_1,stand
2,2,1424696633918,1424696631923288855,-5.995087,0.653549,8.204376,a,nexus4,nexus4_1,stand
3,3,1424696633919,1424696631928385290,-5.942718,0.676163,8.128204,a,nexus4,nexus4_1,stand
4,4,1424696633929,1424696631933420691,-5.991516,0.641647,8.135345,a,nexus4,nexus4_1,stand
5,5,1424696633929,1424696631938456091,-5.965332,0.629745,8.128204,a,nexus4,nexus4_1,stand
6,6,1424696633938,1424696631943522009,-5.991516,0.635696,8.162720,a,nexus4,nexus4_1,stand
7,7,1424696633939,1424696631948496374,-5.915344,0.630936,8.105591,a,nexus4,nexus4_1,stand
8,8,1424696633951,1424696631953592810,-5.984375,0.694016,8.067505,a,nexus4,nexus4_1,stand
9,9,1424696633952,1424696631960428747,-5.937958,0.715439,8.090118,a,nexus4,nexus4_1,stand


In [4]:
HHAR_watch['User'].value_counts()

e    1614999
g    1587697
i    1557737
b    1548768
f    1384531
a    1362520
h    1342301
d    1337124
c    1326798
Name: User, dtype: int64

In [5]:
HHAR_watch_refined = HHAR_watch.dropna()
HHAR_watch_refined['User'].value_counts()

i    1342007
b    1320280
e    1310856
g    1278593
d    1223857
h    1223093
a    1218871
c    1188186
f    1173532
Name: User, dtype: int64

In [6]:
HHAR_watch['Device'].unique()

array(['nexus4_1', 'nexus4_2', 's3_1', 's3_2', 's3mini_1', 's3mini_2',
       'samsungold_1', 'samsungold_2'], dtype=object)

In [7]:
HHAR_watch['Model'].unique()

array(['nexus4', 's3', 's3mini', 'samsungold'], dtype=object)

In [8]:
HHAR_watch['gt'].unique()

array(['stand', nan, 'sit', 'walk', 'stairsup', 'stairsdown', 'bike'],
      dtype=object)

In [9]:
HHAR_watch['User'].unique()

array(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i'], dtype=object)

In [11]:
HHAR_watch=HHAR_watch.dropna()
HHAR_watch['gt'].unique()

array(['stand', 'sit', 'walk', 'stairsup', 'stairsdown', 'bike'],
      dtype=object)

In [12]:
HHAR_watch['gt'] = HHAR_watch['gt'].astype('category')
HHAR_watch['gt'].unique()

['stand', 'sit', 'walk', 'stairsup', 'stairsdown', 'bike']
Categories (6, object): ['bike', 'sit', 'stairsdown', 'stairsup', 'stand', 'walk']

In [13]:
HHAR_watch['labels']=HHAR_watch['gt'].cat.codes
HHAR_watch['labels'].unique()



array([4, 1, 5, 3, 2, 0], dtype=int8)

In [3]:
def sliding_window(time_series, width, step, order='F'):
    w = np.hstack([time_series[i:1 + i - width or None:step] for i in range(0, width)])
    result = w.reshape((int(len(w) / width), width), order='F')
    if order == 'F':
        return result
    else:
        return np.ascontiguousarray(result)

def calc_normalization(data):
    num_instances, num_time_steps, num_features = data.shape
    data = np.reshape(data, (num_instances, -1))
    scaler.fit(data)
#     mean, std = (np.array([np.mean(x) for x in X_train], dtype=np.float32), np.array([np.std(x) for x in X_train], dtype=np.float32))
    return scaler
def apply_normalization(data, scaler):
#     scaler = StandardScaler()
    num_instances, num_time_steps, num_features = data.shape
    data = np.reshape(data, (num_instances, -1))
    norm_data = scaler.transform(data)
#     debug_here()
#     data = (data - mean) / (std + 1e-5)
    norm_data[np.isnan(norm_data)] = 0
    norm_data = np.reshape(norm_data, (num_instances, num_time_steps, num_features))
    return norm_data


In [14]:
def HHAR_data_generator(HHAR_):
    full_data={}
    seq_length = 128
    shifting_step = 128
    num_variables = 3
    
    HHAR_=HHAR_watch.dropna()
    HHAR_['gt'] = HHAR_['gt'].astype('category')
    HHAR_['labels']=HHAR_['gt'].cat.codes
    # print(HHAR_['labels'].unique())
    
    # drop irrelevant columns 
    HHAR_refined = HHAR_.drop(columns=['Index', 'Arrival_Time', 'Creation_Time', 'Model','Device' , 'gt'])
#     user_index = 0
    
    for user_name, user_data in  HHAR_refined.groupby('User'): #group by each user, or can ignore it, use 1 big file
        data, labels = [], []
        
        # looping through data from each class per user
        for class_name, class_data in user_data.groupby('labels'):
            col_index=0
            # slicing data to 128 with no overlap 
            sliced_data = np.empty((int(class_data.shape[0]/seq_length),seq_length, num_variables))
            for column in class_data[['x', 'y', 'z']]:
                column_data = class_data[column]
                sliced_data[:,:,col_index] = sliding_window(column_data.values, seq_length,shifting_step, 'T')
                col_index +=1
           # append data 
            data.append(sliced_data)
            # generate labels
            class_labels = np.empty(sliced_data.shape[0])
            class_labels.fill(class_name)
            labels.append(class_labels.astype(int))
            
        # data and labels for each users 
        array_user_data= np.concatenate(data, axis=0 )
        array_user_labels= np.concatenate(labels, axis=0 )        
        
        print('data: ', array_user_data.dtype)
        print('labels: ',array_user_labels.dtype)
        # Stratified train, validation, test split of the data 
        X_train, X_test, y_train, y_test = train_test_split(array_user_data, array_user_labels,  stratify=array_user_labels,  test_size=0.3,random_state=1)
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=1)

        # Data normalization 
        # Calculate mean and standard deviation based on train
        scaler = calc_normalization (X_train)
        
        # Apply normalization 
        X_train = apply_normalization(X_train,scaler)
        X_val = apply_normalization(X_val,scaler)
        X_test = apply_normalization(X_test,scaler)
        
        # prepare samples
        train_data = {'samples':X_train, 'labels':y_train}
        val_data   = {'samples':X_val, 'labels':y_val}
        test_data  = {'samples':X_test, 'labels':y_test}
        
        #gyroscope  accelerometer
        torch.save(train_data, f'../HHAR_train_val_test/train_wa.pt') #split it according to the user; also can merge 0-8 as one big file
        torch.save(test_data, f'../HHAR_train_val_test/test_wa.pt')
        torch.save(val_data,  f'../HHAR_train_val_test/val_wa.pt')
        # user_index+=1

        # print(user_index, ":", len(train_data["samples"])+len(test_data["samples"]))
    

In [16]:
# concat accelerometer and gyroscope
HHAR_a=pd.read_csv(f'{data_dir}/Watch_accelerometer.csv')
# HHAR_watch_g=pd.read_csv(f'{data_dir}/Phones_gyroscope.csv')

# HHAR_watch=pd.concat([HHAR_watch_a,HHAR_watch_g])
HHAR_data_generator(HHAR_a)

/var/folders/vp/5q7d1xdj6xn18sv64g2jrcdr0000gn/T/ipykernel_9357/1587053602.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HHAR_['gt'] = HHAR_['gt'].astype('category')
/var/folders/vp/5q7d1xdj6xn18sv64g2jrcdr0000gn/T/ipykernel_9357/1587053602.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HHAR_['labels']=HHAR_['gt'].cat.codes


data:  float64
labels:  int64
data:  float64
labels:  int64
data:  float64
labels:  int64
data:  float64
labels:  int64
data:  float64
labels:  int64
data:  float64
labels:  int64
data:  float64
labels:  int64
data:  float64
labels:  int64
data:  float64
labels:  int64


In [45]:
HHAR_watch_a=HHAR_watch_a.dropna()
HHAR_watch_a['User'].value_counts()

i    1342007
b    1320280
e    1310856
g    1278593
d    1223857
h    1223093
a    1218871
c    1188186
f    1173532
Name: User, dtype: int64

In [25]:
HHAR_watch_a['User'].value_counts()

e    620188
f    578939
c    507779
b    497544
a    496424
d    440868
g    161141
h    141538
i     96541
Name: User, dtype: int64

In [28]:
HHAR_watch_g['User'].value_counts()

e    567252
f    535078
b    497348
c    478461
a    453610
d    420959
h    116592
g     92428
i     43703
Name: User, dtype: int64

In [52]:
data_folder = '/Users/lizliao/Downloads/GitFYP_experiment/Dataset/'
#gyroscope  accelerometer

train_data = torch.load(data_folder + 'HHAR Processed Data/HHAR_SA/train_0.pt')
test_data = torch.load(data_folder + 'HHAR Processed Data/HHAR_SA/test_0.pt')

X_train = train_data['samples']
y_train = train_data['labels']
# y_train.min(), y_train.max()
X_test = test_data['samples']
y_test = test_data['labels']

len(X_train)+len(X_test)

1523

In [40]:
train_data = torch.load(data_folder + 'HHAR Processed Data/HHAR_w_users/accelerometer/train_0.pt')
test_data = torch.load(data_folder + 'HHAR Processed Data/HHAR_w_users/accelerometer/test_0.pt')

X_train = train_data['samples']
y_train = train_data['labels']
# y_train.min(), y_train.max()
X_test = test_data['samples']
y_test = test_data['labels']
len(X_train)+len(X_test)

10380

In [56]:
data_dir = '../Activity recognition exp'
HHAR_ = pd.read_csv(f'{data_dir}/Phones_accelerometer.csv')

In [59]:
HHAR_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13062475 entries, 0 to 13062474
Data columns (total 10 columns):
 #   Column         Dtype  
---  ------         -----  
 0   Index          int64  
 1   Arrival_Time   int64  
 2   Creation_Time  int64  
 3   x              float64
 4   y              float64
 5   z              float64
 6   User           object 
 7   Model          object 
 8   Device         object 
 9   gt             object 
dtypes: float64(3), int64(3), object(4)
memory usage: 996.6+ MB


In [63]:
HHAR_ = HHAR_ .dropna()
HHAR_['User'].value_counts()

i    1342007
b    1320280
e    1310856
g    1278593
d    1223857
h    1223093
a    1218871
c    1188186
f    1173532
Name: User, dtype: int64